In [58]:
import torch
import numpy as np
import sys
import os
import pandas as pd
from torch.utils.data import DataLoader, Subset, ConcatDataset
sys.path.insert(0, '../Utils/')
from train_utils import ModelReadyData_diffdim, convert_to_dict, prediction_sepatt, BCE_Weighted_Reg, BCE_Weighted_Reg_focal, compute_loss_for_all_labels_sepatt

In [81]:
proj_dir = '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/'
feature_path_tcga = os.path.join(proj_dir + 'intermediate_data/Old_5_model_ready_data', "TCGA_PRAD","IMSIZE250_OL0", 'feature_uni2', 'TFT' + str(0.9))

data_dir =  proj_dir  +"/intermediate_data/Old_5_model_ready_data/OPX/IMSIZE250_OL0/feature_uni2/TFT0.9/OPX_info.pth"
data_dir2 =  proj_dir  +"/intermediate_data/Old_5_model_ready_data/OPX/IMSIZE250_OL0/feature_uni2/TFT0.9/OPX_ids.pth"
data_dir3 =  proj_dir  +"/intermediate_data/Old_5_model_ready_data/OPX/IMSIZE250_OL0/feature_uni2/TFT0.9/OPX_data.pth"

In [83]:
tcga_data = torch.load(feature_path_tcga + '/TCGA_data.pth')
tcga_ids = torch.load(feature_path_tcga + '/TCGA_ids.pth')
tcga_info  = torch.load(feature_path_tcga + '/TCGA_info.pth')

In [84]:
#count labels in tcga
tcga_label_counts = [dt[1] for dt in tcga_data] 
tcga_label_counts = torch.concat(tcga_label_counts)
count_ones = (tcga_label_counts == 1).sum(dim=0)
print(count_ones)
perc_ones = count_ones/tcga_label_counts.shape[0] * 100
formatted_numbers = [f"{x.item():.1f}" for x in perc_ones]
print(formatted_numbers) #["AR","PTEN","RB1","TP53"

tensor([ 2, 34, 13,  3, 53,  0,  4])
['0.4', '7.6', '2.9', '0.7', '11.9', '0.0', '0.9']


In [87]:
len(tcga_data)

447

In [86]:
round(2/len(tcga_data)*100,1)

0.4

In [46]:
SELECTED_FOLD = 0
folder_name_ids = 'uni1/TrainOL100_TestOL0_TFT' + str(0.9)  + "/"
train_val_test_id_path =  os.path.join(proj_dir + 'intermediate_data/Old_6_Train_TEST_IDS', folder_name_ids)
train_test_val_id_df = pd.read_csv(train_val_test_id_path + "train_test_split.csv")
train_ids_all = list(train_test_val_id_df.loc[train_test_val_id_df['FOLD' + str(SELECTED_FOLD)] == 'TRAIN', 'SAMPLE_ID'])
test_ids_all = list(train_test_val_id_df.loc[train_test_val_id_df['FOLD' + str(SELECTED_FOLD)] == 'TEST', 'SAMPLE_ID'])
val_ids_all = list(train_test_val_id_df.loc[train_test_val_id_df['FOLD' + str(SELECTED_FOLD)] == 'VALID', 'SAMPLE_ID'])

In [51]:
all_ids = train_ids_all + test_ids_all + val_ids_all
len(all_ids)

203

In [54]:
[x for x in ids if x not in all_ids]

['OPX_087']

In [80]:
ids = torch.load(data_dir2)
data = torch.load(data_dir3)
info = torch.load(data_dir)

KeyboardInterrupt: 

In [61]:
#Train:
inc_idx = [ids.index(x) for x in all_ids]
opx_data = Subset(data, inc_idx)
opx_ids =  list(Subset(ids, inc_idx))
opx_info = list(Subset(info, inc_idx))

In [76]:
#count labels in train
train_label_counts = [dt[1] for dt in opx_data]
train_label_counts = torch.concat(train_label_counts)
count_ones = (train_label_counts == 1).sum(dim=0)
print(count_ones)
perc_ones = count_ones/train_label_counts.shape[0] * 100
formatted_numbers = [f"{x.item():.1f}" for x in perc_ones]
ALL_LABELS = ["AR","MMR (MSH2, MSH6, PMS2, MLH1, MSH3, MLH3, EPCAM)2","PTEN","RB1","TP53","TMB_HIGHorINTERMEDITATE","MSI_POS"]

print(ALL_LABELS)
print(formatted_numbers)

tensor([20, 33, 42, 22, 74, 18, 17])
['AR', 'MMR (MSH2, MSH6, PMS2, MLH1, MSH3, MLH3, EPCAM)2', 'PTEN', 'RB1', 'TP53', 'TMB_HIGHorINTERMEDITATE', 'MSI_POS']
['9.9', '16.3', '20.7', '10.8', '36.5', '8.9', '8.4']


In [74]:
round(18/len(opx_data)*100,1)

8.9